### Learning More About Pandas by Using a Weighted Average

在 Python 的開發中，Pandas 是我們的好朋友，他是一個免費的開源資料庫，由 Wes McKinney 在2008年開發的，主要是為了方便和直觀地處理關係型或標籤型數據而製作的。它提供了各種數據結構和操作，用於處理數字數據和時間序列。這個庫是建立在NumPy庫之上的，所以通常使用上會一同進行，Pandas的速度很快，它對用戶來說有很高的性能和生產力，所以目前被廣泛應用在資料科學上。

Pandas包含多個內置函數，如sum、mean、max、min等，我們可以應用到 DataFrame 或分組數據中。他可以提高你在數據處理和分析方面的效率。而在這邊加權平均數是一個很好的使用例子，因為它是一個容易理解但有用的公式。我發現，在查看某些數據集合時，它比簡單的平均數更直觀。在pandas中建立一個加權平均函數是比較簡單的，但是當與其他 pandas 函數(groupby) 結合起來時，會起到不可思議的作用。本文將討論為什麼你會選擇使用加權平均數來觀察你的數據的基本原理，然後介紹如何在pandas中構建和使用這個函數。本文所展示的基本原則將有助於在pandas中建立更複雜的分析，也應該有助於理解如何在pandas中處理分組數據。

- https://pbpython.com/weighted-average.html

### 為什麼使用加權平均？
下面一個簡單的例子說明了為什麼加權平均數可以成為一個有用的統計方法。下面表格顯示了3個不同的顧客為同一產品支付的價格和數量。如果有人問，我們鞋子的平均價格是多少？鞋子價格的簡單平均數將是 (300+200+150=$216.67) 雖然這是一個準確的平均數值，但對於理解我們的平均銷售價格來說，這在直覺上是沒有意義的。

In [1]:
import pandas as pd
import numpy as np

dt  = {'Customer Price': ["Huang", "Chun", "Ming"], 
       'Shoe Price': [300, 200, 150],
       'Shoe Quantity': [20, 100, 225]}

pd.DataFrame(dt)

,Customer Price,Shoe Price,Shoe Quantity
0,Huang,300,20
1,Chun,200,100
2,Ming,150,225


如果我們想用平均數來預測收入，這就特別具有挑戰性。但如果看一下這些數字，你可以看到我們以<200美元的價格出售的鞋子遠遠多於超過200美元的。因此，216.67美元的平均值並不能準確反映市場上的真實平均售價。更有用的是，根據購買數量對這些價格進行加權。這樣的平均能代表所有客戶的購買模式。

- 加權平均數可以這樣計算。
- (鞋子價格*數量)的總和 / (鞋子的總和
- (300∗20) + (200∗100) + (150∗225) / (20+100+225)=$173.19

由於我們絕大部分的鞋子都是在200美元到150美元之間銷售，這個數字比簡單的平均數更準確地代表了我們產品的整體平均價格。

### 在Pandas中計算加權平均數

加權的概念很簡單，但在pandas中計算起來可能有點困難，因為你需要兩個值：要平均的值（鞋價）和重量（鞋量）。所以當你在做複雜的數據分組時，在pandas 中實現它可能會有點不直觀。在許多不同的情況下使用加權平均數就會變得非常容易。此外，建立這個功能並在各種情況下使用它的過程對於培養你日常的pandas數據操作技能應該是很有用的。

In [2]:
sales = pd.read_excel("data/estimate.xlsx")
sales.head()

,Account,Name,State,Rep,Manager,Current_Price,Quantity,New_Product_Price
0,714466,Trantow-Barrows,MN,Craig Booker,Debra Henley,500,100,550
1,737550,"Fritsch, Russel and Anderson",MN,Craig Booker,Debra Henley,600,90,725
2,146832,Kiehn-Spinka,TX,Daniel Hilton,Debra Henley,225,475,255
3,218895,Kulas Inc,TX,Daniel Hilton,Debra Henley,290,375,300
4,412290,Jerde-Hilpert,WI,John Smith,Debra Henley,375,400,400


在我們的例子數據中，我們有一堆賬戶信息，包括當前的價格和數量，以及預測的新產品價格。如果我們想確定一個簡單的平均值，我們可以使用內置的函數來輕鬆計算。

In [3]:
print(sales["Current_Price"].mean())
print(sales["New_Product_Price"].mean())

405.4166666666667
447.0833333333333


### 計算加權平均值

In [4]:
# determine a simple mean
print((sales["Current_Price"] * sales["Quantity"]).sum() / sales["Quantity"].sum())
print((sales["New_Product_Price"] * sales["Quantity"]).sum() / sales["Quantity"].sum())

342.5406871609403
374.6383363471971


In [5]:
# Use np.average to simplify the formula
print(np.average(sales["Current_Price"], weights=sales["Quantity"]))
print(np.average(sales["New_Product_Price"], weights=sales["Quantity"]))

342.5406871609403
374.6383363471971


加權平均公式並不復雜，但它很冗長。當我們對數據進行分組時，它也將很難使用。如果我們建立一個用加權平均數對數據進行分組的函數，就會簡單得多。

In [6]:
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()
    
print(wavg(sales, "Current_Price", "Quantity"))
print(wavg(sales, "New_Product_Price", "Quantity"))

342.5406871609403
374.6383363471971


### 用加權平均法(Weighted Average) 對數據分組
groupby常用於總結數據。例如，如果我們想看一下經理的Current_Price的平均值，用groupby就很簡單。

In [7]:
sales.groupby("Manager").apply(wavg, "Current_Price", "Quantity")

Manager
Debra Henley     340.665584
Fred Anderson    344.897959
dtype: float64

In [8]:
sales.groupby("Manager").apply(wavg, "New_Product_Price", "Quantity")

Manager
Debra Henley     372.646104
Fred Anderson    377.142857
dtype: float64

In [9]:
sales.groupby("State").apply(wavg, "New_Product_Price", "Quantity")

State
CA    446.428571
MN    632.894737
NV    325.000000
TX    274.852941
WA    610.000000
WI    440.000000
dtype: float64

- 你也可以通過多個標准進行分組。

In [10]:
sales.groupby(["Manager", "State"]).apply(wavg, "New_Product_Price", "Quantity")

Manager        State
Debra Henley   MN       632.894737
               TX       274.852941
               WI       440.000000
Fred Anderson  CA       446.428571
               NV       325.000000
               WA       610.000000
dtype: float64

### 多重聚合 (multiple aggregation)

我想介紹的最後一個項目是對數據進行多重聚合的能力。例如，如果我們想得到一些列的平均值，一個列的中位數和另一個列的總和，我們可以通過定義一個帶有列名和要調用的聚合函數的字典來實現。然後，我們在分組的數據上調用 agg

In [11]:
f = {'New_Product_Price': ['mean'],'Current_Price': ['median'], 'Quantity': ['sum', 'mean']}
sales.groupby("Manager").agg(f)

New_Product_Price Current_Price Quantity            
                           mean        median      sum        mean
Manager                                                           
Debra Henley         471.666667         437.5     1540  256.666667
Fred Anderson        422.500000         375.0     1225  204.166667

- 類似的方法，將多個自定義函數組合成一個單一的數據框架

In [12]:
data_1 = sales.groupby("Manager").apply(wavg, "New_Product_Price", "Quantity")
data_2 = sales.groupby("Manager").apply(wavg, "Current_Price", "Quantity")

In [13]:
summary = pd.DataFrame(data=dict(s1=data_1, s2=data_2))
summary.columns = ["New Product Price","Current Product Price"]
summary.head()

,New Product Price,Current Product Price
Manager,,
Debra Henley,372.646104,340.665584
Fred Anderson,377.142857,344.897959


### 使用Numpy
正如我上面提到的，Numpy有一個平均函數，它可以接受一個權重列表併計算出一個加權平均數。
下面是如何使用它來獲得所有未分組數據的加權平均數的方法。

In [14]:
np.average(sales["Current_Price"], weights=sales["Quantity"])

342.5406871609403

- 使用一個lambda函數，讓它與分組數據一起工作

In [15]:
sales.groupby("Manager").apply(lambda x: np.average(x['New_Product_Price'], weights=x['Quantity']))

Manager
Debra Henley     372.646104
Fred Anderson    377.142857
dtype: float64